# NYCHealth Coronavirus (COVID-19) data
### Original datasource: https://github.com/nychealth/coronavirus-data

In [ ]:
import pandas as pd
import requests
import json
import re
import csv
from airflow.models import Variable
from datetime import datetime

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
REPO_NAME = 'coronavirus-data'
REPO_OWNER = 'nychealth'
API_ENDPOINT = 'https://api.github.com/repos/'
RAW_DATA_ENDPOINT = 'https://raw.githubusercontent.com/'

In [ ]:
GIT_USER = Variable.get('GIT_USER', default_var=None)
GIT_TOKEN = Variable.get('GIT_TOKEN', default_var=None)

In [ ]:
session = requests.session()
session.auth = (GITUSER, GITTOKEN)

In [ ]:
response = session.get(f'{api_endpoint}/{repo_owner}/{repo_name}/commits')
commit_shas = list(map(lambda commit: (commit['commit']['author']['date'], commit['sha']), json.loads(response.text)))

In [ ]:
df = []
for (date, commit) in commit_shas:
    
    response = session.get(f'{raw_data_endpoint}/{repo_owner}/{repo_name}/{commit}/tests-by-zcta.csv')
    if response.status_code == 200:
        
        csv_dict = csv.DictReader(response.text.split('\n'))
        for row in csv_dict:
            
            row.update({'Date': date})  # add commit_date as field: Date
            df.append(row)


In [ ]:
df = pd.DataFrame(df)

df["Last_Updated_Date"] = datetime.datetime.utcnow()
df['Last_Reported_Date'] = df['Date'] == df['Date'].max()

In [ ]:
df.head()

In [ ]:
df.to_csv(output_folder + "NYC_HEALTH_TESTS.csv")